In [688]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.callbacks import EarlyStopping

In [689]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import scale

In [690]:
f9 = r'/home/william/Dropbox/Kaggle/Soccer/season9.csv'
f10 = r'/home/william/Dropbox/Kaggle/Soccer/season10.csv'
f11 = r'/home/william/Dropbox/Kaggle/Soccer/season11.csv'
f12 = r'/home/william/Dropbox/Kaggle/Soccer/season12.csv'
f13 = r'/home/william/Dropbox/Kaggle/Soccer/season13.csv'
f14 = r'/home/william/Dropbox/Kaggle/Soccer/season14.csv'
f15 = r'/home/william/Dropbox/Kaggle/Soccer/season15.csv'
f16 = r'/home/william/Dropbox/Kaggle/Soccer/season16.csv'


df9 = pd.read_csv(f9)
df10 = pd.read_csv(f10)
df11 = pd.read_csv(f11)
df12 = pd.read_csv(f12)
df13 = pd.read_csv(f13)
df14 = pd.read_csv(f14)
df15 = pd.read_csv(f15)
df16 = pd.read_csv(f16)

df_train = df9.append([df10,df11,df12,df13,df14,df15,df16],ignore_index=True)

df_train = df_train.sample(frac=1)

In [691]:
df_train

,stage,B365H,B365D,B365A,Home/Away win,competence_difference
2956,30,2.30,3.40,3.40,1,-0.066667
2610,34,2.30,3.75,3.10,0,-0.617647
2376,10,2.05,3.40,4.20,1,0.500000
2304,3,4.75,3.80,1.80,-1,-1.000000
2317,4,1.73,4.00,5.00,0,-0.250000
1843,33,2.50,3.40,3.00,1,-0.121212
2537,26,1.44,4.75,8.00,1,0.423077
1923,3,1.44,5.00,7.50,1,0.000000
3,1,3.60,3.25,2.10,-1,0.000000
135,14,1.91,3.50,4.20,0,0.500000


In [700]:
X = np.array(scale(df_train.drop('Home/Away win',axis=1),axis=0),dtype='float')

In [702]:
y = np.array(df_train['Home/Away win'],dtype='int')

winning_odds = X[np.arange(X.shape[0]),-3-y]

y = np.vstack((y,winning_odds)).T

new_y = np.zeros((y.shape[0],3))
for n in np.arange(y.shape[0]):
    new_y[n,int(y[n,0])+1] = y[n,1]
    
y = np.copy(new_y)

In [703]:
train = 2200
valid = 2500

In [704]:
train_X = np.copy(X[:train,:])/5
valid_X = np.copy(X[train:valid,:])/5
test_X = np.copy(X[valid:,:])/5

train_y = np.copy(y[:train])
valid_y = np.copy(y[train:valid])
test_y = np.copy(y[valid:])

## Time to start model building!

In [705]:
import keras.backend as K
def loss_fun(y_true,y_pred):
    return -K.sum(y_pred*y_true-y_pred,axis=-1)
    
def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

    

In [706]:
def profits(model,test_X,test_y):
    predictions = model.predict(test_X.reshape((-1,5)))
    all_profits = predictions*test_y-predictions
    return np.sum(all_profits)/np.sum(predictions)

In [714]:
model = Sequential()

model.add(Dense(256,input_shape=(5,)))
model.add(Activation('sigmoid'))
model.add(Dropout(0.1))


#model.add(Dense(32))
#model.add(Activation('sigmoid'))
#model.add(Dropout(0.4))


model.add(Dense(3))
model.add(Activation('softmax'))


In [715]:
model.compile(optimizer='adam',loss=mean_squared_error,metrics=['accuracy'])

In [716]:
cb = EarlyStopping(monitor='val_loss', patience=100, verbose=0)

In [717]:
history = model.fit(train_X,train_y,validation_data=(valid_X,valid_y),
          batch_size=16,epochs=1000,verbose=0)

In [723]:
profits(model,test_X,test_y)

-0.98109184109966208

In [721]:
model.evaluate(test_X,test_y)

 32/540 [>.............................] - ETA: 0s

[0.33248030388796773, 0.33148148103996561]

In [722]:
plt.plot(history.history['acc'])
plt.show()

In [727]:
model.predict(test_X)

array([[  4.67561930e-01,   4.72054094e-01,   6.03839643e-02],
       [  4.44277227e-01,   5.55704355e-01,   1.84125965e-05],
       [  3.29572111e-01,   3.02188843e-01,   3.68238986e-01],
       ..., 
       [  1.89546943e-01,   8.10453057e-01,   6.15917928e-10],
       [  1.81576550e-01,   8.18423450e-01,   1.43760748e-09],
       [  3.38863224e-01,   6.61136806e-01,   1.28653177e-08]], dtype=float32)